# ElGamal Cryptosystem

## Key Generation

Steps:

* A trusted party chooses a prime number $p$ and an element of the field $g$ (of large order, ideally a generator).
* The party that generates the keys draws a private key $a$<$p$ and computes $A=g^a (\text{mod }p)$. Publishes $A$ as the public key

The public key is $A$ and the private key $a$.

In [1]:
from crypto import GeneratePrimeGeneratorPair

size_bits = 16

p, g = GeneratePrimeGeneratorPair(size_bits)

print(f"p = {p}")
print(f"g = {g}")

p = 40039
g = 24594


In [2]:
from random import randrange



print(f"PublicKey: {A}")
print(f"PrivateKey: {a}")

PublicKey: 17890
PrivateKey: 35717


## Encryption function

The party that wants to send a message $m$ and knows the public parameters knows $p$, $g$ and $A$ (the latter one is the public key). To encrypt we need a random ephemeral key $k$ to compute the two parts of the ciphertext

$$c_1=g^k (\text{mod }p)$$
$$c_2=mA^k (\text{mod }p)$$

The ciphertext is the tuple ($c_1$, $c_2$).

In [3]:

print(f"m: {m}")
print(f"k: {k}")
print(f"c: {c}")

m: 22525
k: 29927
c: (22992, 36516)


## Decryption function

The decryption function computes the ciphertext $c$=$($c_1$, $c_2$) to the original message $m$. 

$$m = (c_1^a)^{-1}*c_2 (\text{mod }p)$$


In [4]:
from crypto import InverseFermat


print(f"message decrypted: {m2}")

message decrypted: 22525


Let's check analitically that the previous equation holds. On the one hand we have $(c_1^a)^{-1}$:

$$(c_1^a)^{-1}(\text{mod }p)=(g^{ka})^{-1}(\text{mod }p)=(A^{k})^{-1}(\text{mod }p)$$

Also

$$c_2(\text{mod }p)=m*A^k(\text{mod }p)$$

if we multiply both we get

$$(c_1^a)^{-1}*c_2 (\text{mod }p)=(A^{k})^{-1}*m*A^k(\text{mod }p)=m(\text{mod }p)$$

proving that the previous decryption is correct.

In [5]:
from typing import Tuple

def ElGamalKeyGenerator(size: int = 64):
    '''
    Implementation of El Gamal Cryptosystem
    This function generates plublic and private keys
    Input:
        size: size in bits of the field
    Output:
        PublicKey: (A, g, p)
        PrivateKey: (sk, p)
    '''
    p, g = GeneratePrimeGeneratorPair(size)
    sk = randrange(2, p-1)
    A = pow(g, sk, p)

    # Return public key and private key
    return (A, g, p), (sk, p)

def ElGamalEncrypt(m: int, PublicKey: Tuple[int]):
    '''
    Encrypts a message m using the ElGamal public key
    Input:
        m: message (An integer message) (mod p)
        PublicKey: A tuple (A, g, p)
    Output:
        c: tuple (c1, c2) encrypted message 
    '''
    A, g, p = PublicKey[0], PublicKey[1], PublicKey[2]
    k = randrange(2, p-1)

    return (pow(g, k, p), pow(A, k, p)*m%p)


def ElGamalDecrypt(c: Tuple[int, int], PrivateKey: Tuple[int]):
    '''
    Decrypts a ciphertext m using the El Gamnal private key
    Input:
        c: tuple (c1, c2) the ciphertext of the message
        PublicKey: A tuple (sk, p)
    Output:
        m: Decrypted message
    '''
    c1, c2 = c[0], c[1]
    sk, p = PrivateKey[0], PrivateKey[1]
    x = InverseFermat(pow(c1, sk, p), p)
    return (x * c2)%p

## What is the difficulty of breaking this code?

Recall that the attacker just knows the public key $A=g^a (\text{mod }p)$ along with the prime number $p$ and generator $g$. In order to crack the cipher, he has to crack the discrete logarithm problem. We can say that ElGamal cryptosystem is as difficult to crack as to solve the discrete logarithm problem:

find $a$ given $A$, $g$ and $p$.

$$A=g^a \textit{(mod p)}$$

In [6]:
# generating private-public pair
size_bits = 16
p, g = GeneratePrimeGeneratorPair(size_bits)

# generate secrets
a = randrange(p)
A = pow(g, a, p)

print(f"p:{p}")
print(f"g:{g}")
print(f"a (secret_key):{a}")
print(f"A (public_key):{A}")

p:45587
g:17267
a (secret_key):31030
A (public_key):36589


In [7]:

    
print(f"The secret key a: {a}")
print(f"The cracked secret key: {a_}")

The secret key a: 31030
The cracked secret key: 31030


## Known ciphertext attack

Let's imagine this scenario: Alice wants Bob and Charlie to send her messages so she creates a key pair using ElGamal ($A$, $a$) for public-private keys. Imagine that Alice also acts as an oracle for the users, this is, if Charlie sends her a ciphertext, she can provide the decrypted message to him. This is no harm in principle as Charlie knew his message beforehand. Alice is forbidden to decrypt the same message twice, i.e. if she is asked to decrypt the same ciphertext twice, she won't answer as maybe that party intercepted the original ciphertext.

Let's say Bob intercepted a ciphertext $c=(c_1, c_2)=(g^k, mA^k)$ that Charlie sent to Alice and he wants to know what Charlie has to say to Alice (get $m$). He can't send that ciphertext to Alice for decryption as Alice will know that Bob is trying to cheat. But he can do the following...


Bob only knows the ciphertext $c_1$ and $c_1$ and creates a random message and a ephemeral key $m^{\prime}$, $k^{\prime}$ and computes the following ciphertext to send to Alice for decryption:

$$c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m)$$

Alice will decrypt this, since this ciphertext $c^{\prime}$ is different from the original ciphertext $c$. Call the decrypted ciphertext $m^{\prime\prime}$. This is what Bob gets as the decryption of $c^{\prime}$ from Alice.

At this point Bob knows $c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m)$, the plaintext for $c^{\prime}$, i.e $m^{\prime\prime}$ and obviously the public key $A$. 

$$c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m)=(g^{k+k^{\prime}},mm^{\prime}A^{k+k^{\prime}})$$

Therefore he determines that

$$m^{\prime\prime}=m*m^{\prime} (\text{mod }p)$$

or equivalently:

$$m=m^{\prime\prime}*(m^{\prime})^{-1} (\text{mod }p)$$

In [8]:
# generating private-public pair
size_bits = 256
p, g = GeneratePrimeGeneratorPair(size_bits)

# generate secrets
a = randrange(p)
A = pow(g, a, p)

print(f"p:\n\t{p}")
print(f"g:\n\t{g}")
print(f"a (secret_key):\n\t{a}")
print(f"A (public_key):{A}\n\t")

p:
	95538085779234286748815328809779361831369700006442621500594893171934599809907
g:
	53282043043702641238899761920673987926113805935408082718060741817514428971610
a (secret_key):
	82302175902270481841060901613186850079609122270774914860118623625415571792480
A (public_key):595559075994467165283963750992636949038553877170707406583224148588779088071
	


In [9]:
# message and random k
m = b"Private msg: Charlie->Alice"
print(f"message: {m}")
assert 8*len(m)<=size_bits, f"Message too large to encrypt in one block"

m_int = int.from_bytes(m, "big")
print(f"message in integer form {m_int}")

message: b'Private msg: Charlie->Alice'
message in integer form 33093944081837745256705073718405113179401926955399334845200556901


In [10]:
# Charlie encrypts message


print(f"m:\n\t{m}")
print(f"m_int:\n\t{m_int}")
print(f"k:\n\t{k}")
print(f"c:\n\t{c}")

m:
	b'Private msg: Charlie->Alice'
m_int:
	33093944081837745256705073718405113179401926955399334845200556901
k:
	19926483725148623043867463051877123780720943286283664526255655762637010532432
c:
	(6525674650038950516040397185405780128501626843753688742990564434121950252490, 89427035188553269723778794077003776210165977217672260512162145826069470511944)


Bob intercepts message from Charlie to Alice so he knows c1 and c2 but cannot ask Alice to decrypt this. However he can compute $c^{\prime}$:

$$c^{\prime}=(c_1g^{k^{\prime}}, c_2A^{k^{\prime}}m)=(g^{k+k^{\prime}},mm^{\prime}A^{k+k^{\prime}})=(c_1^{\prime},c_2^{\prime} )$$.

In [11]:
# choose a random m_p and k_p on the field


# compute the ciphertext


Bob sends $c^{\prime}$ to Alice to decrypt and get $m^{\prime \prime}$

In [12]:

print(f"m_pp:\n\t{m_pp}")

m_pp:
	79797905553768412717595586225314572104465631706344031001780547044515161188229


Bob knows now $m^{\prime}$ and $m^{\prime \prime}$ so he can calculate the oringial message that Bob sent to Alice:

$$m=m^{\prime\prime}*(m^{\prime})^{-1} (\text{mod }p)$$

In [13]:
m_int_recovered = m_pp*InverseFermat(m_p, p)%p
m_recovered = m_int_recovered.to_bytes(len(m), 'big')

print(f"message:\n\t{m}")
print(f"message recovered:\n\t{m_recovered}")

message:
	b'Private msg: Charlie->Alice'
message recovered:
	b'Private msg: Charlie->Alice'
